In [1]:
from k_means_clustering import kmeans

In [2]:
%%time
output_img = kmeans('D:/work/kmeans/images/2024_01_30_cutted.tif', "D:/work/kmeans/result_images/2024_01_30_OPTICS_cutted.geotif", "OPTICS")

(185712, 3)


KeyboardInterrupt: 